In [45]:
from google.cloud import firestore
import gcsfs
import pickle5 as pickle
import textblob
import nltk

In [22]:
%env GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/each/covid-news/keys/covid-news-a9d0938c837d.json"

env: GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/each/covid-news/keys/covid-news-a9d0938c837d.json"


In [23]:
DB = firestore.Client.from_service_account_json("/Users/marcelcanhisares/Development/each/covid-news/keys/covid-news-a9d0938c837d.json")

In [31]:
KEYWORDS = ["corona", "covid", "pandemia", "SARS", "covid-19"]

STOP_WORDS = [ 'a', 'à', 'agora', 'ainda', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ampla', 'amplas', 'amplo', 'amplos', 'ante', 'antes', 'ao', 'aos', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'através', 'cada', 'coisa', 'coisas', 'com', 'como', 'contra', 'contudo', 'da', 'daquele', 'daqueles', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'do', 'dos','e','é','ela','elas','ele','eles','em','entre','era','eram','éramos','essa','essas','esse','esses','esta','está','estamos','estão','estas','estava','estavam','estávamos','este','esteja','estejam','estejamos','estes','esteve','estive','estivemos','estiver','estivera','estiveram','estivéramos','estiverem','estivermos','estivesse','estivessem','estivéssemos','estou','eu','foi','fomos','for','fora','foram','fôramos','forem','formos','fosse','fossem','fôssemos','fui','há','haja','hajam','hajamos','hão','havemos','havia','hei','houve','houvemos','houver','houvera','houverá','houveram','houvéramos','houverão','houverei','houverem','houveremos','houveria','houveriam','houveríamos','houvermos','houvesse','houvessem','houvéssemos','isso','isto','já','lhe','lhes','mais','mas','me','mesmo','meu','meus','minha','minhas','muito','na','não','nas','nem','no','nos','nós','nossa','nossas','nosso','nossos','num','numa','o','os','ou','para','pela','pelas','pelo','pelos','por','qual','quando','que','quem','são','se','seja','sejam','sejamos','sem','ser','será','serão','serei','seremos','seria','seriam','seríamos','seu','seus','só','sobre','somos','sou','sua','suas','também','te','tem','têm','temos','tenha','tenham','tenhamos','tenho','ter','terá','terão','terei','teremos','teria','teriam','teríamos','teu','teus','teve','tinha','tinham','tínhamos','tive','tivemos','tiver','tivera','tiveram','tivéramos','tiverem','tivermos','tivesse','tivessem','tivéssemos','tu','tua','tuas','um','uma','você','vocês','vos']


In [24]:
fs = gcsfs.GCSFileSystem(token="/Users/marcelcanhisares/Development/each/covid-news/keys/covid-news-a9d0938c837d.json", project="covid-news-291320")

In [25]:
def getClassifier():
    directory = "covid-news-291320.appspot.com"
    filename = "Classificador.pickle"
    with fs.open(directory + '/' + filename, 'rb') as handle:
        return pickle.load(handle)

In [53]:
document = DB.collection('articles').document('0006126928ebb6f907e325da8a67b3944a6d1f07540639d03d1268f0edaca0f3')

In [55]:
document.get().to_dict()

{'publishedAt': '2020-10-23T18:27:30Z',
 'category': None,
 'title': 'Coronavírus: por que a covid-19 mata tanto? - BBC News Brasil',
 'url': 'https://www.bbc.com/portuguese/geral-54658766',
 'author': 'https://www.facebook.com/bbcnews',
 'description': 'Por que a biologia do coronavírus faz com que ele represente uma ameaça à vida humana?',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_portuguese/D505/production/_115033545_gettyimages-1226314512.jpg',
 'articleId': '0006126928ebb6f907e325da8a67b3944a6d1f07540639d03d1268f0edaca0f3'}

In [26]:
collection = DB.collection('articles').stream()

In [27]:
articlesDocs = []
for document in collection:
    articlesDocs.append(document.to_dict())

In [30]:
def removeArticleSource(text:str):
	index = text.rfind('-')	
	return text[:index - 1] if index > 1 else text

In [33]:
def removeStopWords(text: str):
	splitedResults = []	
	splited = text.split()
	for word in splited:
		if word not in STOP_WORDS and word not in KEYWORDS:
			splitedResults.append(word)
	return splitedResults

In [32]:
def stemWords(words):	
	stemmedWords = []
	stemmer = nltk.stem.RSLPStemmer()
	for word in words:
		stemmedWords.append(stemmer.stem(word))
	return stemmedWords

In [36]:
def classifyText(classifier, text):
    words = removeStopWords(text)
    stemmedWords = stemWords(words)
    textToClassify = " ".join(stemmedWords)
    return classifier.classify(textToClassify)

In [ ]:
classifier = getClassifier()

In [56]:
def updateDocument(item_id, field_updates):
    doc = DB.collection('articles').document(item_id)
    doc.update(field_updates)

In [57]:
for article in articlesDocs:
    if article["category"] is None:
        field_updates = {}        
        field_updates["title"] = removeArticleSource(article["title"])
        field_updates["category"] = classifyText(classifier, article["title"])
        updateDocument(article["articleId"], field_updates)

In [52]:
articlesDocs[1]

{'category': None,
 'title': 'Coronavírus: por que a covid-19 mata tanto? - BBC News Brasil',
 'url': 'https://www.bbc.com/portuguese/geral-54658766',
 'author': 'https://www.facebook.com/bbcnews',
 'description': 'Por que a biologia do coronavírus faz com que ele represente uma ameaça à vida humana?',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_portuguese/D505/production/_115033545_gettyimages-1226314512.jpg',
 'articleId': '0006126928ebb6f907e325da8a67b3944a6d1f07540639d03d1268f0edaca0f3',
 'publishedAt': '2020-10-23T18:27:30Z'}

In [46]:
classifyText(classifier, articlesDocs[1]["title"])

'politica'